Import

In [27]:
# Import libraries
import numpy as np
from scipy import ndimage
from imageio import imread
import cv2
from matplotlib import pyplot as plt
import sys
from sklearn.cluster import KMeans
import os
import queue
import math
import random
from IPython.display import clear_output
import warnings
import pygame, time
warnings.filterwarnings("ignore")

Extract Frames

In [28]:
# Setup for object tracking

if not os.path.isdir(os.path.join(os.getcwd(), 'frames')):
    os.mkdir("frames")
else:
    print('frames already exists')

if not os.path.isdir(os.path.join(os.getcwd(), 'composite')):
    os.mkdir("composite")
else:
    print('composite already exists')

framenumber = 955 #0
omovie = cv2.VideoCapture('Opt1-MarionetteMovements.mov')
frame_height = omovie.get(cv2.CAP_PROP_FRAME_HEIGHT)
frame_width = omovie.get(cv2.CAP_PROP_FRAME_WIDTH)

# Extract the frames from original video
# while(1):
    # ret, frame = omovie.read()
    # if not ret:
        # break
    # print('Extracting: %d' % framenumber)
    # clear_output(wait=True)
    # cv2.imwrite('frames/%d.tif' % framenumber, frame)
    # framenumber += 1
# omovie.release()
# 

frames already exists


Get Centroids

In [29]:
def img_show(img):
    new_img = img.copy()
    img_R = img[:, :, 2]
    img_B = img[:, :, 0]
    new_img[:, :, 2] = img_B
    new_img[:, :, 0] = img_R
    new_img = np.uint8(new_img)

    plt.figure()
    plt.imshow(new_img)
    plt.axis('off')
    plt.show()

def preprocessing_padding_helper(img, kernel_size):
    pad_size = int(kernel_size / 2)  
    padding_2_dims = ((pad_size, pad_size), (pad_size, pad_size))
    img_padded = np.pad(img, padding_2_dims, 'constant', constant_values=0)
    return img_padded

def labelBodyParts(centroids, k):
    if len(centroids) < k:
        return []
    
    listOfX = []
    for x in range(len(centroids)): listOfX.append(centroids[x][0])
    listOfY = []
    for y in range(len(centroids)): listOfY.append(centroids[y][1])

    maxX = max(listOfX)
    minX = min(listOfX)
    maxY = max(listOfY)
    minY = min(listOfY)
    avgX = (maxX+minX)/2
    avgY = (maxY+minY)/2

    # Sort
    findBody = queue.PriorityQueue()
    for i in range(len(centroids)):
        dist = math.sqrt((centroids[i][0] - avgX)**2 + (centroids[i][1] - avgY)**2)
        findBody.put((dist, centroids[i]))
    body = findBody.get(0)[1]

    leftArm = None
    rightArm = None
    getArmLeft = queue.PriorityQueue() 
    getArmRight = queue.PriorityQueue()
    for i in range(len(centroids)):
        if (centroids[i][1] < body[1]):
            if (centroids[i][0] < body[0]):
                getArmLeft.put((centroids[i][0] - body[0], centroids[i])) # problem because of negatives
            else:
                getArmRight.put((body[0] - centroids[i][0], centroids[i]))

    if not getArmLeft.empty():
        leftArm = getArmLeft.get(0)[1]
    if not getArmRight.empty():
        rightArm = getArmRight.get(0)[1]

    leftLeg = None
    rightLeg = None
    getLegRight = queue.PriorityQueue() 
    getLegLeft = queue.PriorityQueue() 
    for i in range(len(centroids)):
        if (centroids[i][1] > body[1]):
            getLegRight.put((body[0] - centroids[i][0], centroids[i]))
            getLegLeft.put((centroids[i][0] - body[0], centroids[i]))

    if not getLegRight.empty():
        rightLeg = getLegRight.get(0)[1]
    if not getLegLeft.empty():
        leftLeg = getLegLeft.get(0)[1]

    final_centroids = [leftArm, rightArm, body, leftLeg, rightLeg]
    return final_centroids

def applyRedMask(img, k):
    hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # Red filter
    lower_red = np.array([0, 50, 150])
    upper_red = np.array([20, 255, 255])
    red_mask = cv2.inRange(hsv_img, lower_red, upper_red)
    red_img = cv2.convertScaleAbs(np.array(255 - red_mask))

    # Noise filtering
    kernel = np.ones((3, 3), np.uint8)
    dilated_img = cv2.dilate(red_img, kernel, iterations=3)
    se = np.ones((6, 6))
    denoised_image = cv2.adaptiveThreshold(dilated_img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 55, 5)
    eroded_img = cv2.erode(denoised_image, se, iterations=2)

    # Contours 
    contours, hierarchy = cv2.findContours(eroded_img, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_NONE)

    list_of_centroids = []
    for contour in contours:
        # Calculate the moments of the contour
        M = cv2.moments(contour)
        
        # Calculate the centroid coordinates
        if M["m00"] != 0:
            cx = int(M["m10"] / M["m00"])
            cy = int(M["m01"] / M["m00"])
        else:
            cx, cy = 0, 0

        list_of_centroids.append((cx, cy))

    centroids_final = labelBodyParts(list_of_centroids, k)

    return centroids_final
        
def findCentroids():
    centeroid_list = []

    for r in range(framenumber):
        current_frame = np.array(cv2.imread('frames/%d.tif' % r))
        height, width, channels = current_frame.shape
        result_frame = np.zeros((height, width, channels), dtype=np.uint8)
    
        curr_centroids = applyRedMask(current_frame, 5)

        # Draw circles at the centers of the largest contours
        isEmpty = False
        if len(curr_centroids) == 5:

            for center in curr_centroids:
                if (center == None):
                    isEmpty = True
            if not isEmpty:
                centeroid_list.append((curr_centroids))
            #plt.imshow(result_frame, cmap='gray')
            #plt.axis('off')  # Turn off axis labels and ticks
            #plt.show()

        # only append to processed frame if there is 5 points 
        #for c in range (0, len(centeroid_list)):
        #    cv2.circle(result_frame, (int(centeroid_list[c][0]), int(centeroid_list[c][1])), 10, (0, 0, 255), -1)

    return centeroid_list

centroids = findCentroids()
print(centroids)

[[(182, 118), (284, 76), (255, 174), (230, 262), (262, 241)], [(181, 134), (285, 80), (256, 177), (234, 263), (265, 240)], [(187, 136), (286, 80), (255, 176), (238, 260), (266, 237)], [(173, 133), (286, 79), (245, 178), (239, 254), (264, 234)], [(194, 125), (285, 81), (243, 179), (248, 245), (248, 245)], [(199, 115), (285, 85), (239, 180), (244, 246), (244, 246)], [(200, 109), (285, 89), (234, 181), (237, 249), (237, 249)], [(193, 110), (284, 92), (231, 181), (226, 253), (226, 253)], [(178, 122), (284, 95), (228, 183), (204, 261), (236, 245)], [(166, 142), (284, 97), (225, 183), (193, 260), (231, 245)], [(154, 162), (284, 103), (220, 184), (186, 259), (228, 244)], [(148, 167), (286, 111), (216, 186), (182, 260), (226, 245)], [(142, 158), (283, 110), (212, 188), (182, 260), (227, 244)], [(139, 145), (283, 159), (209, 189), (184, 258), (226, 244)], [(142, 128), (283, 159), (226, 176), (187, 256), (228, 243)], [(153, 113), (283, 159), (206, 188), (190, 254), (231, 241)], [(156, 105), (283

Run and Create the game

In [32]:
pygame.init()

# Create a circle image
circle_radius = 5
circle_color = (255, 0, 0)  # Red
circle = pygame.Surface((circle_radius * 2, circle_radius * 2), pygame.SRCALPHA)
pygame.draw.circle(circle, circle_color, (circle_radius, circle_radius), circle_radius)

# Get frame size
current_frame = np.array(cv2.imread('frames/%d.tif' % 0))
height, width, channels = current_frame.shape

# Make background & set screen size
screen = pygame.display.set_mode((width, height))
bg = pygame.image.load('img/background.jpg')
bg2 = pygame.image.load('img/background2.png')
background = pygame.transform.smoothscale(bg, (width, height))
background2 = pygame.transform.smoothscale(bg2, (width, height))
pygame.display.set_caption("Cooking Game")

# Character pictures
cookie = pygame.image.load('img/cookie1.png')
cookieJar = pygame.image.load('img/cookieJar2.png')
left_hand = pygame.image.load('img/chefLeftHand.png')
right_hand = pygame.image.load('img/chefRightHand.png')
body = pygame.image.load('img/chefBody.png')
left_foot = pygame.image.load('img/chefLeftFoot.png')
right_foot = pygame.image.load('img/chefRightFoot.png')

player_width, player_height = 30, 35
        
# All the rules of my game
class GameRules():
    def __init__(self, screen, limit):
        self.screen = screen
        self.game_over = False
        self.score_limit = limit
        self.score = 0
        self.won = False
        self.generate_world()
        # text info
        self.font = pygame.font.SysFont('Bauhaus 93', 60)
        self.inst_font = pygame.font.SysFont('Bauhaus 93', 30)
        self.color = pygame.Color("brown")

    # create and add player and cookie to the screen
    def generate_world(self):
        self.player = CookieJar(width // 2, height - player_height, player_width, player_height)
        self.cookie = Cookie(centroids[0][0][0], centroids[0][0][1])

    # Adds cookies to the game
    def add_cookie(self, x, y):
        self.cookie = Cookie(x, y)

    # Checks if user caught a cookie, increases score if caught
    def ball_caught(self):
        if pygame.Rect.colliderect(self.cookie.rect, self.player.rect):
            self.score = self.score + 1
            self.cookie.disappear()
    
    # Checks if the player hit the chef, if it does moves to the corner
    def hit_chef(self):
        if pygame.Rect.colliderect(self.cookie.rect, self.body.rect):
            self.player.move_corner()
    
    # Using centroids from the moneky creates the chef
    def add_body_parts(self, parts):
        self.curr_parts = parts
        self.left_hand = BodyPart(parts[0][0], parts[0][1], left_hand)
        self.right_hand = BodyPart(parts[1][0], parts[1][1], right_hand)
        self.body = BodyPart(parts[2][0], parts[2][1], body)
        self.left_foot = BodyPart(parts[3][0], parts[3][1], left_foot)
        self.right_foot = BodyPart(parts[4][0], parts[4][1], right_foot)
        self.left_arm = Limbs(parts[0][0], parts[0][1], parts[2][0], parts[2][1], pygame.Color("white"), 20)
        self.right_arm = Limbs(parts[1][0], parts[1][1], parts[2][0], parts[2][1], pygame.Color("white"), 20)
        self.left_leg = Limbs(parts[3][0], parts[3][1], parts[2][0], parts[2][1], pygame.Color("black"), 15)
        self.right_leg = Limbs(parts[4][0], parts[4][1], parts[2][0], parts[2][1], pygame.Color("black"), 15)

    def player_move(self):
        keys = pygame.key.get_pressed()
        # for player controls
        if keys[pygame.K_UP]:
            if self.player.rect.top > 0:
                self.player.move_up()
        if keys[pygame.K_DOWN]:
            if self.player.rect.bottom < height:
                self.player.move_down()
        if keys[pygame.K_LEFT]:
            if self.player.rect.left > 0:
                self.player.move_left()
        if keys[pygame.K_RIGHT]:
            if self.player.rect.right < width:
                self.player.move_right()

    def show_score(self):
        score = str(self.score)
        score = self.font.render(score, True, self.color)
        self.screen.blit(score, ((width // 4) * 3.7, 20))
    
    # Returns if you won or not
    def game_end(self):
        return self.won

    def update(self):
        self.show_score()
        self.player.update(self.screen)        
        if self.score == self.score_limit:
            self.won = True
        self.cookie.update(self.screen)
        self.left_arm.update(self.screen)
        self.right_arm.update(self.screen)
        self.left_leg.update(self.screen)
        self.right_leg.update(self.screen)
        self.left_hand.update(self.screen)
        self.right_hand.update(self.screen)
        self.body.update(self.screen)
        self.left_foot.update(self.screen)
        self.right_foot.update(self.screen)
        self.ball_caught()
        self.hit_chef()

# Player
class CookieJar:
    def __init__(self, x, y, width, height):
        self.x = x
        self.y = y
        self.rect = pygame.Rect(self.x, self.y, width, height)
        self.player_speed = 10
        self.score = 0
        self.image = cookieJar

    def move_up(self):
        self.rect.y -= self.player_speed

    def move_down(self):
        self.rect.y += self.player_speed
    
    def move_left(self):
        self.rect.x -= self.player_speed
    
    def move_right(self):
        self.rect.x += self.player_speed

    def move_corner(self):
        self.rect.x = self.rect.width
        self.rect.y =  height - self.rect.height

    def update(self, screen):
        screen.blit(self.image, self.rect)

# Chefs hands, feet and center
class BodyPart:
    def __init__(self, x, y, img):
        self.x = x
        self.y = y
        self.radius = 13
        self.image = img
        self.rect = self.image.get_rect(center=(x, y))
        self.speed = random.randint(5, 15)

    def update(self, screen):
        screen.blit(self.image, self.rect)

# Chefs arms and legs
class Limbs:
    def __init__(self, start_x, start_y, end_x, end_y, c, w):
        self.start_x = start_x
        self.start_y = start_y
        self.end_x = end_x
        self.end_y = end_y
        self.color = c
        self.width = w
    
    def update(self, screen):
        dx = self.end_x - self.start_x
        dy = self.end_y - self.start_y
        angle = math.degrees(math.atan2(dy, dx))
        length = math.sqrt(dx**2 + dy**2)
        pygame.draw.line(screen, self.color, (self.start_x, self.start_y), (self.end_x, self.end_y), self.width)

class Cookie:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.radius = 13
        self.image = cookie
        self.rect = self.image.get_rect(center=(x, y))
        self.speed = random.randint(5, 15)
    
    # Checks if theres not a cookie on the screen
    def cookie_out(self):
        if self.y > height - self.radius:
            return True
        else:
            return False
    # Makes cookie disappear so that it doesn't mess with the score
    def disappear(self):
        self.x = 1000
        self.y = 1000

    # Cookies fall at random speed
    def cookie_fall(self):
        self.rect.y = self.y
        self.y += self.speed
        self.screen = screen

    def update(self, screen):
        self.cookie_fall()
        screen.blit(self.image, self.rect)


class CookingGame(): 
    # Initialize everything
    def __init__(self, screen):
        self.screen = screen
        self.FPS = pygame.time.Clock()
        # Initial 5 body part points (of the monkey)
        self.circle1_x = centroids[0][0][0] 
        self.circle1_y = centroids[0][0][1]
        self.circle2_x = centroids[0][1][0] 
        self.circle2_y = centroids[0][1][1]
        self.circle3_x = centroids[0][2][0] 
        self.circle3_y = centroids[0][2][1]
        self.circle4_x = centroids[0][3][0]  
        self.circle4_y = centroids[0][3][1]
        self.circle5_x = centroids[0][4][0]  
        self.circle5_y = centroids[0][4][1]
        self.index = 0
        self.time_since_last_switch = 0
        self.target_fps = 30
        self.centroid_switch_time = 33 #Time Interval = Total Time / Number of Switches = 1000 ms / 30 = 33.33 ms per switch
        self.elapsed_time = 0

    def draw(self):
        pygame.display.flip()

    def main(self):
        playing = False
        game_over = False
        game_rules1 = GameRules(self.screen, 3)  
        game_rules2 = GameRules(self.screen, 5)
        myfont=pygame.font.SysFont("Bauhaus 93", 40)
        level1 = True
        game_2over = False

        # Waiting to start screen
        while (playing==False and not game_over):
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()
            self.screen.blit(background, (0, 0))
            nlabel=myfont.render("Ready to cook? (enter to start)", 1, (255, 255, 255))
            keys = pygame.key.get_pressed()
            if keys[pygame.K_RETURN]:
                self.elapsed_time=0
                playing=True
            screen.blit(nlabel,(15, 250))
            pygame.display.flip()

        # Playing screen 1
        while playing and level1 and not game_over:
            self.screen.blit(bg, (0, 0))
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()

            if game_rules1.game_end():
                self.elapsed_time = 0
                level1 = False
            elif self.elapsed_time > 30000:
                self.elapsed_time = 0
                game_over = True

            dt = self.FPS.tick(self.target_fps)

            self.elapsed_time += self.FPS.get_time()
            self.time_since_last_switch += dt

            # Check if it's time to switch to the next centroid
            if self.time_since_last_switch >= self.centroid_switch_time:
                self.time_since_last_switch = 0
                self.index = (self.index + 1) % len(centroids)  # Cycle through centroids
                game_rules1.add_body_parts(centroids[self.index])
                

            # Place the body parts (using the monkey points)
            if (game_rules1.cookie.cookie_out()):
                hand = random.randint(0, 1)
                game_rules1.add_cookie(centroids[self.index][hand][0], centroids[self.index][hand][1])

            # Get the position from centroids
            self.circle1_x, self.circle1_y = centroids[self.index][0]
            self.circle2_x, self.circle2_y = centroids[self.index][1]
            self.circle3_x, self.circle3_y = centroids[self.index][2] 
            self.circle4_x, self.circle4_y = centroids[self.index][3]
            self.circle5_x, self.circle5_y = centroids[self.index][4]

            pygame.draw.circle(self.screen, circle_color, (self.circle1_x, self.circle1_y), circle_radius)
            pygame.draw.circle(self.screen, circle_color, (self.circle2_x, self.circle2_y), circle_radius)
            pygame.draw.circle(self.screen, circle_color, (self.circle3_x, self.circle3_y), circle_radius)
            pygame.draw.circle(self.screen, circle_color, (self.circle4_x, self.circle4_y), circle_radius)
            pygame.draw.circle(self.screen, circle_color, (self.circle5_x, self.circle5_y), circle_radius)

            # Tracks time
            time = str(self.elapsed_time // 1000)
            nlabel = myfont.render(time, 1, pygame.Color('brown'))


            self.screen.blit(background, (0, 0))
            self.screen.blit(nlabel, (10, 20))

            game_rules1.player_move()
            game_rules1.update()
            self.draw()
            self.FPS.tick(30)

        # Level 2
        while not level1 and not game_2over:
            self.screen.blit(background2, (0, 0))
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()

            if game_rules2.game_end():
                game_2over = True
            elif self.elapsed_time > 30000:
                game_2over = True

            dt = self.FPS.tick(self.target_fps)

            self.elapsed_time += self.FPS.get_time()
            self.time_since_last_switch += dt

            # Check if it's time to switch to the next centroid
            if self.time_since_last_switch >= self.centroid_switch_time:
                self.time_since_last_switch = 0
                self.index = (self.index + 1) % len(centroids)  # Cycle through centroids
                game_rules2.add_body_parts(centroids[self.index])
                

            # Place the body parts (using the monkey points)
            if (game_rules2.cookie.cookie_out()):
                hand = random.randint(0, 1)
                game_rules2.add_cookie(centroids[self.index][hand][0], centroids[self.index][hand][1])

            # Get the position from centroids
            self.circle1_x, self.circle1_y = centroids[self.index][0]
            self.circle2_x, self.circle2_y = centroids[self.index][1]
            self.circle3_x, self.circle3_y = centroids[self.index][2] 
            self.circle4_x, self.circle4_y = centroids[self.index][3]
            self.circle5_x, self.circle5_y = centroids[self.index][4]

            pygame.draw.circle(self.screen, circle_color, (self.circle1_x, self.circle1_y), circle_radius)
            pygame.draw.circle(self.screen, circle_color, (self.circle2_x, self.circle2_y), circle_radius)
            pygame.draw.circle(self.screen, circle_color, (self.circle3_x, self.circle3_y), circle_radius)
            pygame.draw.circle(self.screen, circle_color, (self.circle4_x, self.circle4_y), circle_radius)
            pygame.draw.circle(self.screen, circle_color, (self.circle5_x, self.circle5_y), circle_radius)

            # Tracks time
            time = str(self.elapsed_time // 1000)
            nlabel = myfont.render(time, 1, pygame.Color('brown'))

            self.screen.blit(background2, (0, 0))
            self.screen.blit(nlabel, (10, 20))

            game_rules2.player_move()
            game_rules2.update()
            self.draw()
            self.FPS.tick(30)
        
        # Won case
        while game_rules1.game_end() and game_rules2.game_end():
            green_bg = pygame.Surface((width, height))
            green_bg.fill((0, 255, 0))
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()
            self.screen.blit(green_bg, (0, 0))
            nlabel=myfont.render("You won !!", 1, (255, 255, 255))
            keys = pygame.key.get_pressed()
            if keys[pygame.K_RETURN]:
                playing=True
            screen.blit(nlabel,(150, height // 2))
            pygame.display.flip()
        
        # Lost case
        while game_over and (not game_rules1.game_end() or not game_rules2.game_end()):
            red_bg = pygame.Surface((width, height))
            red_bg.fill((255, 0, 0))
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()
            self.screen.blit(red_bg, (0, 0))
            nlabel=myfont.render("You lost :(", 1, (255, 255, 255))
            keys = pygame.key.get_pressed()
            if keys[pygame.K_RETURN]:
                playing=True
            screen.blit(nlabel,(150, height // 2))
            pygame.display.flip()

# Play the game
if __name__ == "__main__":
    play = CookingGame(screen) # CookingGame
    play.main()


SystemExit: 